In [1]:
from sqlite3 import connect
import pandas as pd
import numpy as np
#encoding=utf-8

class Line:
    def __init__(self,
                 name="",                # 导线名
                 type_name="",           # 导线型号名
                 resistance=0,           # 导线电阻
                 radius=1,               # 导线计算半径
                 equivalent_radius=1,    # 导线等效半径
                 rho=1,                  # 导线电导率
                 mu_r=1,                 # 相对磁导率
                 coordinate_x=0,         # 导线坐标 x
                 coordinate_y=0):        # 导线坐标 y
        self.name = name
        self.type_name = type_name
        self.resistance = resistance
        self.radius = radius
        self.rho = rho
        self.mu_r = mu_r
        self.equivalent_radius = equivalent_radius
        self.coordinate_x = coordinate_x
        self.coordinate_y = coordinate_y

    def set_parameter(self,
                      resistance=0,
                      radius=1,
                      equivalent_radius=1,
                      rho=1,
                      mu_r=1,
                      coordinate_x=0,
                      coordinate_y=0
                      ):
        self.resistance = resistance
        self.radius = radius
        self.rho = rho
        self.mu_r = mu_r
        self.equivalent_radius = equivalent_radius
        self.coordinate_x = coordinate_x
        self.coordinate_y = coordinate_y


class ATLines:
    lines_type = "AT System"

    def __init__(self,
                 name=""
                 ):
        self.name = name
        self.lines = []
        self.lines.append(Line(name="cw1"))
        self.lines.append(Line(name="mw1"))
        self.lines.append(Line(name="pf1"))
        self.lines.append(Line(name="ra1"))
        self.lines.append(Line(name="ra2"))
        self.lines.append(Line(name="pw1"))
        self.lines.append(Line(name="e1"))
        self.lines.append(Line(name="cw2"))
        self.lines.append(Line(name="mw2"))
        self.lines.append(Line(name="pf2"))
        self.lines.append(Line(name="ra3"))
        self.lines.append(Line(name="ra4"))
        self.lines.append(Line(name="pw2"))
        self.lines.append(Line(name="e2"))

    #  def __setattr__(self, file_name=""):
        # for line in lines:
        #     pass
    # def set_accodinater(self):
     #        pass


#if __name__ == '__main__':
    


class TractionTransformer:               # 牵引变压器类
    def __init__(self,
                 name="",
                 alias_name="",          # 牵引变压器型号
                 zs=1+1j,                # 系统内阻抗
                 location=0,             # 安装位置（公里)

                 ):
        self.name = name
        self.alias_name = alias_name
        self.zs = zs
        self.location = location


class AutoTransformer:                    # 自耦变压器类
    def __init__(self,
                 name="",
                 alias_name="",           # 变压器型号
                 zs=1+1j,                 # 漏抗（欧）
                 location=0             # 安装位置(公里）
                 ):
        self.name = name
        self.alias_name = alias_name
        self.zs = zs
        self.location = location


class Locomotive:                        # 机车类
    def __init__(self,
                 name="",                #
                 load=0,                 # 负荷电流（安 ,A）
                 harmonic={},            # 谐波含量 （fn, %）
                 location=0,             # 运行位置 （公里，km）
                 at_upline=1             # 位于上行线=1；位于下行线=0
                 ):
        self.name = name
        self.load = load
        self.harmonic = harmonic
        self.location = location
        self.at_upline = at_upline

class CrossConnection:
    """
    """
    def __init__(self,
                 name="",
                 ):
        self.name = name
        self.to_all = []           # 上下行全并联，( km）
        self.pw1_ra1 = []          # 保护线pw1连钢轨ra1，( km）
        self.e1_ra1 = []           # 综合地线e1连接钢轨ra1，( km）
        self.e1_g = {}             # 综合地线e1连接大地g ，{ km, 欧}
        self.pw2_ra3 = []          # 保护线pw2连钢轨ra3，( km）
        self.e2_ra3 = []          # 综合地线e2连接钢轨ra3，( km）
        self.e2_g = {}             # 综合地线e2连接大地g，{ km, 欧}
        self.ra1_g = {}            # 钢轨ra1连接大地g
        self.ra3_g = {}            # 钢轨ra3连接大地g


class Topology:
    def __init__(self,
                 name="",
                 alias_name="",
                 section_length=30.6,
                 earth_rou=10**6,
                 traction_transformer=[],
                 lines=[],
                 auto_transformers=[],
                 cross_connections=[],
                 locomotive=[]
                 ):
        self.name = name
        self.alias_name = alias_name
        self.section_length = section_length
        self.earth_rou = earth_rou
        self.traction_transformer = traction_transformer
        self.lines = lines
        self.auto_transformers = auto_transformers
        self.cross_connections = cross_connections
        self.locomotive = locomotive

    def set_topology(self,
                     db_file_name="default.db"    # 系统拓扑结构 sqlite数据库文件名
                     ):
        con = connect(db_file_name)#取得数据库连接对象
        cur = con.cursor()#取得数据库游标对象
        #导线选型
        sqlcom_area_line = 'select area_line from star_lines'
        sqlcom_m_line = 'select m_line from star_lines'
        sqlcom_rou_line = 'select rou_line from star_lines'
        sqlcom_i_line = 'select i_line from star_lines'
        sqlcom_cal_r = 'select cal_r from star_lines'
        sqlcom_q_r = 'select q_r from star_lines'
        sqlcom_axis_x = 'select axis_x from star_lines'
        sqlcom_axis_y = 'select axis_y from star_lines'

        df1_sqlcom_area_line = pd.read_sql(sqlcom_area_line,con)
        df1_sqlcom_m_line = pd.read_sql(sqlcom_m_line,con)
        df1_sqlcom_rou_line = pd.read_sql(sqlcom_rou_line,con)
        df1_sqlcom_i_line = pd.read_sql(sqlcom_i_line,con)
        df1_sqlcom_cal_r = pd.read_sql(sqlcom_cal_r,con)
        df1_sqlcom_q_r = pd.read_sql(sqlcom_q_r,con)
        df1_sqlcom_axis_x = pd.read_sql(sqlcom_axis_x,con)
        df1_sqlcom_axis_y = pd.read_sql(sqlcom_axis_y,con)


        df2_sqlcom_area_line = np.array(df1_sqlcom_area_line) #先使用array()将DataFrame转换一下
        df2_sqlcom_m_line = np.array(df1_sqlcom_m_line) #先使用array()将DataFrame转换一下
        df2_sqlcom_rou_line = np.array(df1_sqlcom_rou_line) #先使用array()将DataFrame转换一下
        df2_sqlcom_i_line = np.array(df1_sqlcom_i_line) #先使用array()将DataFrame转换一下
        df2_sqlcom_cal_r = np.array(df1_sqlcom_cal_r) #先使用array()将DataFrame转换一下
        df2_sqlcom_q_r = np.array(df1_sqlcom_q_r) #先使用array()将DataFrame转换一下
        df2_sqlcom_axis_x = np.array(df1_sqlcom_axis_x) #先使用array()将DataFrame转换一下
        df2_sqlcom_axis_y = np.array(df1_sqlcom_axis_y) #先使用array()将DataFrame转换一下

        df3_sqlcom_area_line = df2_sqlcom_area_line.tolist()
        df3_sqlcom_m_line = df2_sqlcom_m_line.tolist()
        df3_sqlcom_rou_line = df2_sqlcom_rou_line.tolist()
        df3_sqlcom_i_line = df2_sqlcom_i_line.tolist()
        df3_sqlcom_cal_r = df2_sqlcom_cal_r.tolist()
        df3_sqlcom_q_r = df2_sqlcom_q_r.tolist()
        df3_sqlcom_axis_x = df2_sqlcom_axis_x.tolist()
        df3_sqlcom_axis_y = df2_sqlcom_axis_y.tolist()


        # print(df3_sqlcom_area_line)
        # print(df3_sqlcom_m_line)
        # print(df3_sqlcom_rou_line)
        # print(df3_sqlcom_i_line)
        # print(df3_sqlcom_cal_r)
        # print(df3_sqlcom_q_r)
        # print(df3_sqlcom_axis_x)
        # print(df3_sqlcom_axis_y)

        #AT变压器
        sqlcom_name_AT = 'select name_AT from star_AT'
        sqlcom_model_AT = 'select model_AT from star_AT'
        sqlcom_location_AT = 'select location_AT from star_AT'
        sqlcom_lou_real = 'select lou_real from star_AT'
        sqlcom_lou_imag = 'select lou_imag from star_AT'

        df1_sqlcom_name_AT = pd.read_sql(sqlcom_name_AT,con)
        df1_sqlcom_model_AT = pd.read_sql(sqlcom_model_AT,con)
        df1_sqlcom_location_AT = pd.read_sql(sqlcom_location_AT,con)
        df1_sqlcom_lou_real = pd.read_sql(sqlcom_lou_real,con)
        df1_sqlcom_lou_imag = pd.read_sql(sqlcom_lou_imag,con)

        df2_sqlcom_name_AT = np.array(df1_sqlcom_name_AT) #先使用array()将DataFrame转换一下
        df2_sqlcom_model_AT = np.array(df1_sqlcom_model_AT) #先使用array()将DataFrame转换一下
        df2_sqlcom_location_AT = np.array(df1_sqlcom_location_AT) #先使用array()将DataFrame转换一下
        df2_sqlcom_lou_real = np.array(df1_sqlcom_lou_real) #先使用array()将DataFrame转换一下
        df2_sqlcom_lou_imag = np.array(df1_sqlcom_lou_imag) #先使用array()将DataFrame转换一下
        

        df3_sqlcom_name_AT = df2_sqlcom_name_AT.tolist()
        df3_sqlcom_model_AT = df2_sqlcom_model_AT.tolist()
        df3_sqlcom_location_AT = df2_sqlcom_location_AT.tolist()
        df3_sqlcom_lou_real = df2_sqlcom_lou_real.tolist()
        df3_sqlcom_lou_imag = df2_sqlcom_lou_imag.tolist()

        #牵引变压器
        sqlcom_name_qianyin = 'select name_qianyin from star_qianyin'
        sqlcom_model_qianyin = 'select model_qianyin from star_qianyin'
        sqlcom_neizukang_qianyin_real = 'select neizukang_qianyin_real from star_qianyin'
        sqlcom_neizukang_qianyin_imag = 'select neizukang_qianyin_imag from star_qianyin'
        sqlcom_location_qianyin = 'select location_qianyin from star_qianyin'

        df1_sqlcom_name_qianyin = pd.read_sql(sqlcom_name_qianyin,con)
        df1_sqlcom_model_qianyin = pd.read_sql(sqlcom_model_qianyin,con)
        df1_sqlcom_neizukang_qianyin_real = pd.read_sql(sqlcom_neizukang_qianyin_real,con)
        df1_sqlcom_neizukang_qianyin_imag = pd.read_sql(sqlcom_neizukang_qianyin_imag,con)
        df1_sqlcom_location_qianyin = pd.read_sql(sqlcom_location_qianyin,con)
        

        df2_sqlcom_name_qianyin = np.array(df1_sqlcom_name_qianyin) #先使用array()将DataFrame转换一下
        df2_sqlcom_model_qianyin = np.array(df1_sqlcom_model_qianyin) #先使用array()将DataFrame转换一下
        df2_sqlcom_neizukang_qianyin_real = np.array(df1_sqlcom_neizukang_qianyin_real) #先使用array()将DataFrame转换一下
        df2_sqlcom_neizukang_qianyin_imag = np.array(df1_sqlcom_neizukang_qianyin_imag) #先使用array()将DataFrame转换一下
        df2_sqlcom_location_qianyin = np.array(df1_sqlcom_location_qianyin) #先使用array()将DataFrame转换一下

        df3_sqlcom_name_qianyin = df2_sqlcom_name_qianyin.tolist()
        df3_sqlcom_model_qianyin = df2_sqlcom_model_qianyin.tolist()
        df3_sqlcom_neizukang_qianyin_real = df2_sqlcom_neizukang_qianyin_real.tolist()
        df3_sqlcom_neizukang_qianyin_imag = df2_sqlcom_neizukang_qianyin_imag.tolist()
        df3_sqlcom_location_qianyin = df2_sqlcom_location_qianyin.tolist()


        #上下行并联线
        #sqlcom_name_1 = 'select name_0_1_2 from star_shangxiaxingbinglianxian'
        #sqlcom_name_2 = 'select name_3_4_5 from star_shangxiaxingbinglianxian'
        sqlcom_location_star_up_down = 'select location_star_up_down from star_up_down'

        #df1_sqlcom_name_1 = pd.read_sql(sqlcom_name_1,con)
        #df1_sqlcom_name_2 = pd.read_sql(sqlcom_name_2,con)
        df1_sqlcom_location_star_up_down = pd.read_sql(sqlcom_location_star_up_down,con)

        #df2_sqlcom_name_1 = np.array(df1_sqlcom_name_1) #先使用array()将DataFrame转换一下
        #df2_sqlcom_name_2 = np.array(df1_sqlcom_name_2) #先使用array()将DataFrame转换一下
        df2_sqlcom_location_star_up_down = np.array(df1_sqlcom_location_star_up_down) #先使用array()将DataFrame转换一下

        #df3_sqlcom_name_1 = df2_sqlcom_name_1.tolist()
        #df3_sqlcom_name_2 = df2_sqlcom_name_2.tolist()
        df3_sqlcom_location_star_up_down = df2_sqlcom_location_star_up_down.tolist()

        # print(df3_sqlcom_name_1)
        # print(df3_sqlcom_name_2)
        # print(df3_sqlcom_location_star_shangxiaxingbinglianxian)

        #pw1和ra1
        sqlcom_location_star_pw1_ra1 = 'select location_star_pw1_ra1 from star_pw1_ra1'

        df1_sqlcom_location_star_pw1_ra1 = pd.read_sql(sqlcom_location_star_pw1_ra1,con)

        df2_sqlcom_location_star_pw1_ra1 = np.array(df1_sqlcom_location_star_pw1_ra1) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_pw1_ra1 = df2_sqlcom_location_star_pw1_ra1.tolist()

        #print(df3_sqlcom_location_star_pw1_ra1)

        #e1和ra1
        sqlcom_location_star_e1_ra1 = 'select location_star_e1_ra1 from star_e1_ra1'
        df1_sqlcom_location_star_e1_ra1 = pd.read_sql(sqlcom_location_star_e1_ra1,con)

        df2_sqlcom_location_star_e1_ra1 = np.array(df1_sqlcom_location_star_e1_ra1) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_e1_ra1 = df2_sqlcom_location_star_e1_ra1.tolist()

        # print(df3_sqlcom_location_star_e1_ra1)

        #e1和g
        sqlcom_location_star_e1_g = 'select location_star_e1_g from star_e1_g'
        sqlcom_Z_star_e1_g = 'select Z_star_e1_g from star_e1_g'

        df1_sqlcom_location_star_e1_g = pd.read_sql(sqlcom_location_star_e1_g,con)
        df1_sqlcom_Z_star_e1_g = pd.read_sql(sqlcom_Z_star_e1_g,con)

        df2_sqlcom_location_star_e1_g = np.array(df1_sqlcom_location_star_e1_g) #先使用array()将DataFrame转换一下
        df2_sqlcom_Z_star_e1_g = np.array(df1_sqlcom_Z_star_e1_g) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_e1_g = df2_sqlcom_location_star_e1_g.tolist()
        df3_sqlcom_Z_star_e1_g = df2_sqlcom_Z_star_e1_g.tolist()

        # print(df3_sqlcom_location_star_e1_g)
        # print(df3_sqlcom_Z_star_e1_g)

        #pw2和ra3
        sqlcom_location_star_pw2_ra3 = 'select location_star_pw2_ra3 from star_pw2_ra3'
        df1_sqlcom_location_star_pw2_ra3 = pd.read_sql(sqlcom_location_star_pw2_ra3,con)

        df2_sqlcom_location_star_pw2_ra3 = np.array(df1_sqlcom_location_star_pw2_ra3) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_pw2_ra3 = df2_sqlcom_location_star_pw2_ra3.tolist()

        #print(df3_sqlcom_location_star_pw2_ra3)

        #e2和ra3
        sqlcom_location_star_e2_ra3 = 'select location_star_e2_ra3 from star_e2_ra3'
        df1_sqlcom_location_star_e2_ra3 = pd.read_sql(sqlcom_location_star_e2_ra3,con)

        df2_sqlcom_location_star_e2_ra3 = np.array(df1_sqlcom_location_star_e2_ra3) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_e2_ra3 = df2_sqlcom_location_star_e2_ra3.tolist()

        #print(df3_sqlcom_location_star_e2_ra3)

        #e2和g
        sqlcom_location_star_e2_g = 'select location_star_e2_g from star_e2_g'
        sqlcom_Z_star_e2_g = 'select Z_star_e2_g from star_e2_g'

        df1_sqlcom_location_star_e2_g = pd.read_sql(sqlcom_location_star_e2_g,con)
        df1_sqlcom_Z_star_e2_g = pd.read_sql(sqlcom_Z_star_e2_g,con)

        df2_sqlcom_location_star_e2_g = np.array(df1_sqlcom_location_star_e2_g) #先使用array()将DataFrame转换一下
        df2_sqlcom_Z_star_e2_g = np.array(df1_sqlcom_Z_star_e2_g) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_e2_g = df2_sqlcom_location_star_e2_g.tolist()
        df3_sqlcom_Z_star_e2_g = df2_sqlcom_Z_star_e2_g.tolist()

        #print(df3_sqlcom_location_star_e2_g)
        #print(df3_sqlcom_Z_star_e2_g)

        #ra1和g
        sqlcom_location_star_ra1_g = 'select location_star_ra1_g from star_ra1_g'
        sqlcom_Z_star_ra1_g = 'select Z_star_ra1_g from star_ra1_g'

        df1_sqlcom_location_star_ra1_g = pd.read_sql(sqlcom_location_star_ra1_g,con)
        df1_sqlcom_Z_star_ra1_g = pd.read_sql(sqlcom_Z_star_ra1_g,con)

        df2_sqlcom_location_star_ra1_g = np.array(df1_sqlcom_location_star_ra1_g) #先使用array()将DataFrame转换一下
        df2_sqlcom_Z_star_ra1_g = np.array(df1_sqlcom_Z_star_ra1_g) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_ra1_g = df2_sqlcom_location_star_ra1_g.tolist()
        df3_sqlcom_Z_star_ra1_g = df2_sqlcom_Z_star_ra1_g.tolist()

        # print(df3_sqlcom_location_star_ra1_g)
        # print(df3_sqlcom_Z_star_ra1_g)

        #ra3和g
        sqlcom_location_star_ra3_g = 'select location_star_ra3_g from star_ra3_g'
        sqlcom_Z_star_ra3_g = 'select Z_star_ra3_g from star_ra3_g'

        df1_sqlcom_location_star_ra3_g = pd.read_sql(sqlcom_location_star_ra3_g,con)
        df1_sqlcom_Z_star_ra3_g = pd.read_sql(sqlcom_Z_star_ra3_g,con)

        df2_sqlcom_location_star_ra3_g = np.array(df1_sqlcom_location_star_ra3_g) #先使用array()将DataFrame转换一下
        df2_sqlcom_Z_star_ra3_g = np.array(df1_sqlcom_Z_star_ra3_g) #先使用array()将DataFrame转换一下

        df3_sqlcom_location_star_ra3_g = df2_sqlcom_location_star_ra3_g.tolist()
        df3_sqlcom_Z_star_ra3_g = df2_sqlcom_Z_star_ra1_g.tolist()

        # print(df3_sqlcom_location_star_ra1_g)
        # print(df3_sqlcom_Z_star_ra1_g)

        #机车
        #sqlcom_1_locomotive = 'select names_star_locomotive from star_locomotive'
        sqlcom_1_locomotive = 'select locomotive1_star_locomotive from star_locomotive'
        sqlcom_2_locomotive = 'select locomotive2_star_locomotive from star_locomotive'
        sqlcom_3_locomotive = 'select locomotive3_star_locomotive from star_locomotive'
        sqlcom_4_locomotive = 'select locomotive4_star_locomotive from star_locomotive'
        sqlcom_5_locomotive = 'select locomotive5_star_locomotive from star_locomotive'
        sqlcom_attribute_locomotive = 'select names_star_locomotive from star_locomotive'
        #sqlcom_Harmonic_content_locomotive = 'select Harmonic_content_locomotive from star_locomotive'
        #sqlcom_location_locomotive = 'select location_locomotive from star_locomotive'

        df1_sqlcom_1_locomotive = pd.read_sql(sqlcom_1_locomotive,con)
        df1_sqlcom_2_locomotive = pd.read_sql(sqlcom_2_locomotive,con)
        df1_sqlcom_3_locomotive = pd.read_sql(sqlcom_3_locomotive,con)
        df1_sqlcom_4_locomotive = pd.read_sql(sqlcom_4_locomotive,con)
        df1_sqlcom_5_locomotive = pd.read_sql(sqlcom_5_locomotive,con)
        df1_sqlcom_5_locomotive = pd.read_sql(sqlcom_5_locomotive,con)
        #df1_sqlcom_location_locomotive = pd.read_sql(sqlcom_location_locomotive,con)
        df1_sqlcom_attribute_locomotive = pd.read_sql(sqlcom_attribute_locomotive,con)
        
        df2_sqlcom_1_locomotive = np.array(df1_sqlcom_1_locomotive) #先使用array()将DataFrame转换一下
        df2_sqlcom_2_locomotive = np.array(df1_sqlcom_2_locomotive) #先使用array()将DataFrame转换一下
        df2_sqlcom_3_locomotive = np.array(df1_sqlcom_3_locomotive) #先使用array()将DataFrame转换一下
        df2_sqlcom_4_locomotive = np.array(df1_sqlcom_4_locomotive) #先使用array()将DataFrame转换一下
        df2_sqlcom_5_locomotive = np.array(df1_sqlcom_5_locomotive) #先使用array()将DataFrame转换一下
        df2_sqlcom_attribute_locomotive = np.array(df1_sqlcom_attribute_locomotive)
        #df2_sqlcom_location_locomotive = np.array(df1_sqlcom_location_locomotive) #先使用array()将DataFrame转换一下

        #df3_sqlcom_1_locomotive = df2_sqlcom_1_locomotive.tolist()
        #df3_sqlcom_2_locomotive = df2_sqlcom_2_locomotive.tolist()
        #df3_sqlcom_3_locomotive = df2_sqlcom_3_locomotive.tolist()
        #df3_sqlcom_4_locomotive = df2_sqlcom_4_locomotive.tolist()
        #df3_sqlcom_5_locomotive = df2_sqlcom_5_locomotive.tolist()
        #df3_sqlcom_location_locomotive = df2_sqlcom_location_locomotive.tolist()
        #print(df2_sqlcom_1_locomotive)
        #print(df2_sqlcom_1_locomotive.shape)
        
        dff_locomotive = np.hstack((df2_sqlcom_1_locomotive,df2_sqlcom_2_locomotive))
        dff_locomotive = np.hstack((dff_locomotive,df2_sqlcom_3_locomotive))
        dff_locomotive = np.hstack((dff_locomotive,df2_sqlcom_4_locomotive))
        dff_locomotive = np.hstack((dff_locomotive,df2_sqlcom_5_locomotive))
        #print(dff_locomotive)
        #print(dff_locomotive.shape[0])
        # print(df3_sqlcom_model_locomotive)
        # print(df3_sqlcom_upanddown_locomotive)
        # print(df3_sqlcom_loadcurrent_locomotive)
        # print(df3_sqlcom_Harmonic_frequency_locomotive)
        # print(df3_sqlcom_Harmonic_content_locomotive)
        # print(df3_sqlcom_location_locomotive)

        
        ###上行##########################################
        cw1=Line(name='cw1')
        cw1.radius=df3_sqlcom_cal_r[0][0]
        cw1.equivalent_radius=df3_sqlcom_q_r[0][0]
        cw1.rho=df3_sqlcom_q_r[0][0]
        cw1.mu_r=1
        cw1.coordinate_x=df3_sqlcom_axis_x[0][0]
        cw1.coordinate_y=df3_sqlcom_axis_y[0][0]
        ####################################
        mw1=Line(name='mw1')
        mw1.radius=df3_sqlcom_cal_r[1][0]
        mw1.equivalent_radius=df3_sqlcom_q_r[1][0]
        mw1.rho=df3_sqlcom_q_r[1][0]
        mw1.mu_r=1
        mw1.coordinate_x=df3_sqlcom_axis_x[1][0]
        mw1.coordinate_y=df3_sqlcom_axis_y[1][0]
        ######################################
        pf1=Line(name='pf1')
        pf1.radius=df3_sqlcom_cal_r[2][0]
        pf1.equivalent_radius=df3_sqlcom_q_r[2][0]
        pf1.rho=df3_sqlcom_q_r[2][0]
        pf1.mu_r=1
        pf1.coordinate_x=df3_sqlcom_axis_x[2][0]
        pf1.coordinate_y=df3_sqlcom_axis_y[2][0]
        ######################################
        ra1=Line(name='ra1')
        ra1.radius=df3_sqlcom_cal_r[3][0]
        ra1.equivalent_radius=df3_sqlcom_q_r[3][0]
        ra1.rho=df3_sqlcom_q_r[3][0]
        ra1.mu_r=1
        ra1.coordinate_x=df3_sqlcom_axis_x[3][0]
        ra1.coordinate_y=df3_sqlcom_axis_y[3][0]
        ######################################
        ra2=Line(name='ra2')
        ra2.radius=df3_sqlcom_cal_r[4][0]
        ra2.equivalent_radius=df3_sqlcom_q_r[4][0]
        ra2.rho=df3_sqlcom_q_r[4][0]
        ra2.mu_r=1
        ra2.oordinate_x=df3_sqlcom_axis_x[4][0]
        ra2.coordinate_y=df3_sqlcom_axis_y[4][0]
        ######################################
        pw1=Line(name='pw1')
        pw1.radius=df3_sqlcom_cal_r[5][0]
        pw1.equivalent_radius=df3_sqlcom_q_r[5][0]
        pw1.rho=df3_sqlcom_q_r[5][0]
        pw1.mu_r=1
        pw1.coordinate_x=df3_sqlcom_axis_x[5][0]
        pw1.coordinate_y=df3_sqlcom_axis_y[5][0]
        ######################################
        e1=Line(name='e1')
        e1.radius=df3_sqlcom_cal_r[6][0]
        e1.equivalent_radius=df3_sqlcom_q_r[6][0]
        e1.rho=df3_sqlcom_q_r[6][0]
        e1.mu_r=1
        e1.coordinate_x=df3_sqlcom_axis_x[6][0]
        e1.coordinate_y=df3_sqlcom_axis_y[6][0]
        ######################################
        ###下行##########################################
        cw2=Line(name='cw2')
        cw2.radius=df3_sqlcom_cal_r[7][0]
        cw2.rho=df3_sqlcom_q_r[7][0]
        cw2.mu_r=1
        cw2.coordinate_x=df3_sqlcom_axis_x[7][0]
        cw2.coordinate_y=df3_sqlcom_axis_y[7][0]
        ####################################
        mw2=Line(name='mw2')
        mw2.radius=df3_sqlcom_cal_r[8][0]
        mw2.equivalent_radius=df3_sqlcom_q_r[8][0]
        mw2.rho=df3_sqlcom_q_r[8][0]
        mw2.mu_r=1
        mw2.coordinate_x=df3_sqlcom_axis_x[8][0]
        mw2.coordinate_y=df3_sqlcom_axis_y[8][0]
        ######################################
        pf2=Line(name='pf2')
        pf2.radius=df3_sqlcom_cal_r[9][0]
        pf2.equivalent_radius=df3_sqlcom_q_r[9][0]
        pf2.rho=df3_sqlcom_q_r[9][0]
        pf2.mu_r=1
        pf2.coordinate_x=df3_sqlcom_axis_x[9][0]
        pf2.coordinate_y=df3_sqlcom_axis_y[9][0]
        ######################################
        ra3=Line(name='ra3')
        ra3.radius=df3_sqlcom_cal_r[10][0]
        ra3.equivalent_radius=df3_sqlcom_q_r[10][0]
        ra3.rho=df3_sqlcom_q_r[10][0]
        ra3.mu_r=1
        ra3.coordinate_x=df3_sqlcom_axis_x[10][0]
        ra3.coordinate_y=df3_sqlcom_axis_y[10][0]
        ######################################
        ra4=Line(name='ra4')
        ra4.radius=df3_sqlcom_cal_r[11][0]
        ra4.equivalent_radius=df3_sqlcom_q_r[11][0]
        ra4.rho=df3_sqlcom_q_r[11][0]
        ra4.mu_r=1
        ra4.coordinate_x=df3_sqlcom_axis_x[11][0]
        ra4.coordinate_y=df3_sqlcom_axis_y[11][0]
        ######################################
        pw2=Line(name='pw2')
        pw2.radius=df3_sqlcom_cal_r[12][0]
        pw2.equivalent_radius=df3_sqlcom_q_r[12][0]
        pw2.rho=df3_sqlcom_q_r[12][0]
        pw2.mu_r=1
        pw2.coordinate_x=df3_sqlcom_axis_x[12][0]
        pw2.coordinate_y=df3_sqlcom_axis_y[12][0]
        ######################################
        e2=Line(name='e2')
        e2.radius=df3_sqlcom_cal_r[13][0]
        e2.equivalent_radius=df3_sqlcom_q_r[13][0]
        e2.rho=df3_sqlcom_q_r[13][0]
        e2.mu_r=1
        e2.coordinate_x=df3_sqlcom_axis_x[13][0]
        e2.coordinate_y=df3_sqlcom_axis_y[13][0]
        ############对lines对象赋值##################################################
        self.lines.append(cw1)
        self.lines.append(mw1)
        self.lines.append(pf1)
        self.lines.append(ra1)
        self.lines.append(ra2)
        self.lines.append(pw1)
        self.lines.append(e1)
        self.lines.append(cw2)
        self.lines.append(mw2)
        self.lines.append(pf2)
        self.lines.append(ra3)
        self.lines.append(ra4)
        self.lines.append(pw2)
        self.lines.append(e2)
        #######对traction_transformer对象赋值############################################
        tractiontransformer_num=len(df3_sqlcom_location_qianyin)
        print('牵引变压器的数量='+str(tractiontransformer_num))
        tractiontransformer_list=[]
        for i in range(0,tractiontransformer_num):
            tractiontransformer_list.append(str(i))       #根据对象的数量形成'代号’列表
        for i in range(0,tractiontransformer_num):
            tractiontransformer_list[i]=TractionTransformer()
            tractiontransformer_list[i].name=df2_sqlcom_name_qianyin[i][0]
            tractiontransformer_list[i].alias_name=df3_sqlcom_model_qianyin[i][0]
            tractiontransformer_list[i].zs_real=df3_sqlcom_neizukang_qianyin_real[i][0]
            tractiontransformer_list[i].zs_imag=df3_sqlcom_neizukang_qianyin_imag[i][0]
            tractiontransformer_list[i].location=df3_sqlcom_location_qianyin[i][0]
            self.traction_transformer.append(tractiontransformer_list[i])
        #################对auto_transformer对象赋值#########################################
        auto_transformer_num=len(df3_sqlcom_location_AT)
        print('AT变压器的数量='+str(auto_transformer_num))
        auto_transformer_list=[]
        for i in range(0,auto_transformer_num):
            auto_transformer_list.append(str(i))       #根据对象的数量形成'代号’列表
        for i in range(0,auto_transformer_num):
            auto_transformer_list[i]=AutoTransformer()
            auto_transformer_list[i].name=df3_sqlcom_name_AT[i][0]
            auto_transformer_list[i].alias_name=df3_sqlcom_model_AT[i][0]
            auto_transformer_list[i].zs_real=df3_sqlcom_lou_real[i][0]
            
            auto_transformer_list[i].zs_imag=df3_sqlcom_lou_imag[i][0]
            
            auto_transformer_list[i].location=df3_sqlcom_location_AT[i][0]
            self.auto_transformers.append(auto_transformer_list[i])
        ########对cross_connections对象赋值###############################
        Crossconnections=CrossConnection()
        #####to_all属性########
        cross_connections_to_all_num=len(df3_sqlcom_location_star_up_down)
        print('上下行并联线的数量='+str(cross_connections_to_all_num))
        for i in range(0,cross_connections_to_all_num):
            Crossconnections.to_all.append(df3_sqlcom_location_star_up_down[i][0])
        ########pw1_ra1属性######
        cross_connections_pw1_ra1_num=len(df3_sqlcom_location_star_pw1_ra1)
        print('pw1与ra1连接线的数量='+str(cross_connections_pw1_ra1_num))
        for i in range(0,cross_connections_pw1_ra1_num):
            Crossconnections.pw1_ra1.append(df3_sqlcom_location_star_pw1_ra1[i][0])
        ######e1_ra1属性###############
        cross_connections_e1_ra1_num=len(df3_sqlcom_location_star_e1_ra1)
        print('e1与ra1连接线的数量='+str(cross_connections_e1_ra1_num))
        for i in range(0,cross_connections_e1_ra1_num):
            Crossconnections.e1_ra1.append(df3_sqlcom_location_star_e1_ra1[i][0])
        ####对e1_g属性##################################################
        cross_connections_e1_g_num=len(df3_sqlcom_location_star_e1_g)
        print('e1与g连接线的数量='+str(cross_connections_e1_g_num))
        for i in range(0,cross_connections_e1_g_num):
            Crossconnections.e1_g[df3_sqlcom_location_star_e1_g[i][0]]=df3_sqlcom_Z_star_e1_g[i][0]
        ####对pw2_ra3属性#############
        cross_connections_pw2_ra3_num=len(df3_sqlcom_location_star_pw2_ra3)
        print('pw2与ra3连接线的数量='+str(cross_connections_pw2_ra3_num))
        for i in range(0,cross_connections_pw2_ra3_num):
            Crossconnections.pw2_ra3.append(df3_sqlcom_location_star_pw2_ra3[i][0])
        ##对e2_g属性##############################################
        cross_connections_e2_g_num=len(df3_sqlcom_location_star_e2_g)
        print('e2与g连接线的数量='+str(cross_connections_e2_g_num))
        for i in range(0,cross_connections_e2_g_num):
            Crossconnections.e2_g[df3_sqlcom_location_star_e2_g[i][0]]=df3_sqlcom_Z_star_e2_g[i][0]
        #####对ra1_g属性#################################################
        cross_connections_ra1_g_num=len(df3_sqlcom_location_star_ra1_g)
        print('ra1与g连接线的数量='+str(cross_connections_ra1_g_num))
        for i in range(0,cross_connections_ra1_g_num):
            Crossconnections.ra1_g[df3_sqlcom_location_star_ra1_g[i][0]]=df3_sqlcom_Z_star_ra1_g[i][0]
        #######对ra3_g属性##############################################
        cross_connections_ra3_g_num=len(df3_sqlcom_location_star_ra3_g)
        print('ra3与g连接线的数量='+str(cross_connections_ra3_g_num))
        for i in range(0,cross_connections_ra3_g_num):
            Crossconnections.ra3_g[df3_sqlcom_location_star_ra3_g[i][0]]=df3_sqlcom_Z_star_ra3_g[i][0]
        #######################
        self.cross_connections.append(Crossconnections)
        #########对机车locomotive对象赋值##########################################################
        locomotive_num=0
        for i in range(5):
            if dff_locomotive[0][i]!=0:
                locomotive_num=locomotive_num+1
            else:
                continue
        print("机车的数量="+str(locomotive_num))
        locomotive_list=[]
        for i in range(locomotive_num):
            locomotive_list.append(str(i))#根据对象的数量形成'代号’列表
            
        for i in range(locomotive_num):
            locomotive_list[i]=Locomotive(name='机车'+str(i+1))
        for i in range(0,locomotive_num):
            locomotive_list[i].location=dff_locomotive[0][i]
            locomotive_list[i].load=dff_locomotive[1][i]
            locomotive_list[i].at_upline=dff_locomotive[2][i]#对每个机车的位置上下行负荷电流赋值
        
        harmonic_num=[]
        for i in range(3,dff_locomotive.shape[0]):
            harmonic_num.append(df2_sqlcom_attribute_locomotive[i][0])
            
        for i in range(0,locomotive_num):
            harmonic_percent=[]
            for j in range(3,dff_locomotive.shape[0]):
                harmonic_percent.append(dff_locomotive[j][i])
            for k in range(0,len(harmonic_num)):
                locomotive_list[i].harmonic[harmonic_num[k]]=harmonic_percent[k]#对每个机车的谐波含量及次数赋值
        
        for i in range(locomotive_num):
            self.locomotive.append(locomotive_list[i])#对topology类中的locomotive对象赋值填充
        
topology = Topology(name="测试供电臂")
#############################################################################################
topology.set_topology(db_file_name="test_calculate.db")
############测试lines对象##################################################
print('====================================================================================')
for i in range(14):
    print(topology.lines[i].name, topology.lines[i].coordinate_x)
print('====================================================================================')
############测试auto_transformers对象#####################################
for i in range(3):
    print(topology.auto_transformers[i].name,topology.auto_transformers[i].alias_name,topology.auto_transformers[i].location,topology.auto_transformers[i].zs_real,topology.auto_transformers[i].zs_imag,type(topology.auto_transformers[i].zs))
print('====================================================================================')
###########测试tractiontransformers对象##################################
print(topology.traction_transformer[0].location)
print('===eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee==================')
print(topology.traction_transformer[0].zs_real,topology.traction_transformer[0].zs_imag)
print('====================================================================================')
##########测试crossconnection对象####################################
print(topology.cross_connections[0].to_all)
print(topology.cross_connections[0].pw1_ra1)
print(topology.cross_connections[0].ra1_g)
print('====================================================================================')
#########测试机车locomotive对象#############################################
for i in range(1):
    print(topology.locomotive[i].harmonic,topology.locomotive[i].name,topology.locomotive[i].load,topology.locomotive[i].location)
print('====================================================================================')
#######测试牵引变压器对象#####################################################
# for i in range(1):
#     print(topology.traction_transformer[i].zs,type(topology.traction_transformer[i].zs))

牵引变压器的数量=1
AT变压器的数量=3
上下行并联线的数量=3
pw1与ra1连接线的数量=11
e1与ra1连接线的数量=62
e1与g连接线的数量=62
pw2与ra3连接线的数量=11
e2与g连接线的数量=62
ra1与g连接线的数量=62
ra3与g连接线的数量=62
机车的数量=1
cw1 0.0
mw1 0.0
pf1 -4400.0
ra1 -755.0
ra2 0
pw1 -3600.0
e1 -4400.0
cw2 5000.0
mw2 5000.0
pf2 9400.0
ra3 4245.0
ra4 5755.0
pw2 8600.0
e2 9400.0
一号 vx 0.0 0.1 0.45 <class 'complex'>
二号 vx 15.802 0.1 0.45 <class 'complex'>
三号 vx 30.6 0.1 0.45 <class 'complex'>
0.0
===eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee==================
2.0 0.0
[0.0, 15.802, 30.6]
[1.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0, 30.0]
{0.0: 50.0, 0.5: 50.0, 2.0: 50.0, 3.0: 50.0, 4.0: 50.0, 5.0: 50.0, 6.0: 50.0, 1.5: 50.0, 8.0: 50.0, 9.0: 50.0, 10.0: 50.0, 11.0: 50.0, 12.0: 50.0, 2.5: 50.0, 14.0: 50.0, 15.0: 50.0, 16.0: 50.0, 17.0: 50.0, 18.0: 50.0, 3.5: 50.0, 20.0: 50.0, 21.5: 50.0, 22.0: 50.0, 23.0: 50.0, 24.0: 50.0, 4.5: 50.0, 22.5: 50.0, 27.0: 50.0, 28.0: 50.0, 26.0: 50.0, 30.0: 50.0, 5.5: 50.0, 27.5: 50.0, 1.0: 50.0, 6.5: 50.0, 28.5: 50.0, 7.5: 50.